## 1.Using Unet for pixel label

In [ ]:
import sys
sys.path.append('/kaggle/input/kaggle-physionet-hengck-demo-00')

from model import *
from dataset import *
from config import *

import matplotlib
import matplotlib.pyplot as plt   

print('import ok!!!')

In [ ]:
net = Net(pretrained=False)
f = torch.load('/kaggle/input/kaggle-physionet-hengck-demo-00/checkpoint.pth', map_location=lambda storage, loc: storage)
state_dict = f['state_dict']
print(net.load_state_dict(state_dict, strict=False))  

net = net.eval()
net.output_type = ['infer']

# dummy data ....
KAGGLE_DIR = '/kaggle/input/physionet-ecg-image-digitization'
class ECGDataset(Dataset):
	def __init__(self):
		self.length = 1000000

		image_id = 31294838
		image_file = f'{KAGGLE_DIR}/train/{image_id}/{image_id}-0001.png'
		image = cv2.imread(image_file, cv2.IMREAD_COLOR_RGB)
		csv_file = f'{KAGGLE_DIR}/train/{image_id}/{image_id}.csv'
		truth_df = pd.read_csv(csv_file)

		crop = make_demo_crop(image)
		lead, horizontal, vertical = make_demo_mask(crop, truth_df)
		print('crop:', crop.shape)
		print('lead:', lead.shape)
		print('horizontal:', horizontal.shape)
		print('vertical:', vertical.shape)
		self.crop = crop
		self.lead = lead
		self.horizontal = horizontal
		self.vertical = vertical

	def __len__(self):
		return self.length

	def __getitem__(self, index):
		out = train_transform(
			image=self.crop.copy(),
			mask=self.lead.copy(),
			horizontal=self.horizontal.copy(),
			vertical=self.vertical.copy(),
		)
		r = {}
		r['image'     ] = torch.from_numpy(np.ascontiguousarray(out['image'].transpose(2,0,1)))  # float32, normalized, CHW
		r['lead'] = torch.from_numpy(out['mask'])
		r['horizontal'] = torch.from_numpy(out['horizontal'])
		r['vertical'  ] = torch.from_numpy(out['vertical'])
		return r

# let's try some image ---------------------------------------------
valid_dataset = ECGDataset()

net = net.cuda()
for trial in range(3):
	print(f'trial:{trial}')
	r = valid_dataset[np.random.choice(len(valid_dataset))]
	batch = {
		'image':r['image'].unsqueeze(0),
	}
	with torch.amp.autocast('cuda', dtype=torch.bfloat16):
		with torch.no_grad():
			output = net(batch)


	if 1:
		image = batch['image'].permute(0,2,3,1).contiguous().data.cpu().numpy()
		horizontal = output['horizontal'].argmax(1).byte().data.cpu().numpy()
		vertical = output['vertical'].argmax(1).byte().data.cpu().numpy()
		lead = output['lead'].argmax(1).byte().data.cpu().numpy()

		B = len(image)
		for b in range(B):
			m = image[b]
			h = horizontal[b]
			v = vertical[b]
			l = lead[b]
			h = color_horizontal(h)
			v = color_vertical(v)
			l = color_lead(l)

			g = cv2.cvtColor(m, cv2.COLOR_RGB2GRAY)
			g = cv2.cvtColor(g, cv2.COLOR_GRAY2RGB)
			overlay = g.copy()
			overlay[h.sum(-1)!=0] =h[h.sum(-1)!=0]
			overlay[v.sum(-1)!=0] =v[v.sum(-1)!=0]
			overlay[l.sum(-1)!=0] =l[l.sum(-1)!=0]

			all = np.vstack([m,l,h,v])

			plt.imshow(overlay)
			plt.show()
			plt.imshow(all)
			plt.show()


## 2.Unwrap Mask to canonical reference

In [ ]:
import numpy as np
import cv2

import torch
import torch.nn.functional as F

import matplotlib.pyplot as plt
import matplotlib


def rectify_image(
	image, gridpoint_xy
):
	H, W = 1700, 2200 #reference size
	H1,W1 = image.shape[:2]
	sparse_map = gridpoint_xy/[[[W1-1,H1-1]]]*2 -1
	sparse_map = torch.from_numpy(np.ascontiguousarray(sparse_map.transpose(2, 0, 1))).unsqueeze(0).float()
	dense_map  = F.interpolate(sparse_map, size=(H, W), mode='bilinear', align_corners=True)
	distort    = torch.from_numpy(np.ascontiguousarray(image.transpose(2, 0, 1))).unsqueeze(0).float()
	rectified = F.grid_sample(
		distort, dense_map.permute(0, 2, 3, 1), mode='bilinear', padding_mode='border', align_corners=False
	)
	rectified = rectified.data.cpu().numpy()
	rectified = rectified[0].transpose(1, 2, 0).astype(np.uint8)
	return rectified

kaggle_dir = '/kaggle/input/physionet-ecg-image-digitization'
processed_dir = '/kaggle/input/hengck23-gridpoint-demo'

all=[]
image_id ='4211091537'
for type_id in ['0005', '0006', '0012']: 
    print('type_id:', type_id)

    image_file = f'{kaggle_dir}/train/{image_id}/{image_id}-{type_id}.png' 
    image = cv2.imread(image_file, cv2.IMREAD_COLOR_RGB)
    gridpoint_xy = np.load(f'{processed_dir}/{image_id}-{type_id}.gridpoint_xy.npy' ) #detected grid point from unet ,,,
    
    rectified = rectify_image(
    	image, gridpoint_xy
    )
    
    plt.imshow(image)
    plt.show()
    plt.imshow(rectified)
    plt.show()
    all.append(rectified[-1000:, 0:1000])

print('compare all results:')
plt.imshow(np.hstack(all), aspect='equal')
plt.show()

## 3.Mask to time series (lead signal)

In [ ]:
import pandas as pd
import cv2
import numpy as np


import matplotlib.pyplot as plt
import matplotlib
import torch
import torch.nn.functional as F

import sys
sys.path.append('/kaggle/input/kaggle-physionet-hengck-demo-00')
from kaggle_metric import *
def compute_one_snr(predict, truth, max_shift=100):
	t = truth
	p = predict
	aligned = align_signals(t, p, max_shift)
	p_signal, p_noise = compute_power(t, aligned)
	snr = compute_snr(p_signal, p_noise)
	snr_db = max(float(10 * np.log10(snr)), -PERFECT_SCORE)
	return snr_db


KAGGLE_DIR = '/kaggle/input/physionet-ecg-image-digitization'
GSY = 39.38095238095238 #height of grid 'square'


#load dummy data
image_id = 31294838
image_file = f'{KAGGLE_DIR}/train/{image_id}/{image_id}-0001.png'
image = cv2.imread(image_file, cv2.IMREAD_COLOR_RGB)
csv_file = f'{KAGGLE_DIR}/train/{image_id}/{image_id}.csv'
truth_df = pd.read_csv(csv_file)

H,W, _ = image.shape


#example signal ------
lead = 'I'
y0 = 708  #zero mV
x0,x1 = 118, 610  #start to end of signal


t    = truth_df[lead].values
mask = ~np.isnan(t)
t    = t[mask]
L = len(t)
print('lead:', lead, L)

overlay = image.copy()
point=[]  #ground truth of pixel label
for i in range(0, L):
	px = int(round(x0 + (i / (L-1)) * (x1 - x0-1)))
	py = int(round(y0 - t[i] * 2 * GSY))
	point.append([px, py])
	overlay[py, px] = [0,255,0]

probility=np.zeros((H,W),np.float32) #probability from unet
for i in range(0, L-1):
	x0, y0 = point[i]
	x1, y1 = point[i+1]
	cv2.line(probility,(x0,y0),(x1,y1), 1,1)#, cv2.LINE_AA)

#make a crop
my,mx=np.where(probility>0)
my0 = my.min()
my1 = my.max()+1
mx0 = mx.min()
mx1 = mx.max()+1
print('crop:', my0, my1, mx0, mx1)

crop = probility[my0:my1, mx0:mx1]
plt.title('probability crop')
plt.imshow(crop,cmap='gray')
plt.show()

plt.title('overlay crop')
plt.imshow(overlay[my0:my1, mx0:mx1],)
plt.show()

In [ ]:
def mask_to_signal(probility, num_sample, zero_mv,  mV_per_pixel):
	H, W = probility.shape
	m = torch.from_numpy(probility).float() #(mask > 0).float()
	idx = torch.arange(H, device=m.device).view(H, 1).to(m.dtype)  # (H,1)
	num = (m * idx).sum(dim=0)  # sum of indices where mask==1, per column -> (W,)
	den = m.sum(dim=0)  # count of ones per column -> (W,)
	non_zero_signal = num / den.clamp_min(1)  # avoid div-by-0 :
	## todo: use nan for missing instead ... then interpolate

	signal = (zero_mv - non_zero_signal) * mV_per_pixel
	resampled = F.interpolate(
		signal.view(1, 1, W), size=num_sample, mode="linear", align_corners=False
	).view(-1) #todo: align_corners=False ???
	resampled = resampled.data.cpu().numpy()
	return resampled


truth = t
num_sample = L
zero_mv = 708-my0  #708 is the bottom of the dc pulse in image
mV_per_pixel = 1/(2*GSY)

resampled = mask_to_signal(crop, num_sample, zero_mv, mV_per_pixel)

snr = compute_one_snr(resampled, truth, max_shift=100)
print('snr', snr)

plt.plot(resampled, label='resampled')
plt.plot(t, label='truth')
plt.legend()
plt.show()

In [ ]:

#another example signal ------
#lead = 'I'
#y0 = 708  #zero mV
#x0,x1 = 118, 610  #start to end of signal

for lead,y0,x0,x1 in [
    ['aVR',708,  610, 1102],
    ['V1', 708, 1102, 1594],
    ['V4', 708, 1594, 2087],
]:

    t    = truth_df[lead].values
    mask = ~np.isnan(t)
    t    = t[mask]
    L = len(t)
    print('lead:', lead, L)
    
    overlay = image.copy()
    point=[]  #ground truth of pixel label
    for i in range(0, L):
    	px = int(round(x0 + (i / (L-1)) * (x1 - x0-1)))
    	py = int(round(y0 - t[i] * 2 * GSY))
    	point.append([px, py])
    	overlay[py, px] = [0,255,0]
    
    probility=np.zeros((H,W),np.float32) #probability from unet
    for i in range(0, L-1):
    	x0, y0 = point[i]
    	x1, y1 = point[i+1]
    	cv2.line(probility,(x0,y0),(x1,y1), 1,1)#, cv2.LINE_AA)
    
    #make a crop
    my,mx=np.where(probility>0)
    my0 = my.min()
    my1 = my.max()+1
    mx0 = mx.min()
    mx1 = mx.max()+1
    print('crop:', my0, my1, mx0, mx1)
    
    crop = probility[my0:my1, mx0:mx1]
    plt.title('probability crop')
    plt.imshow(crop,cmap='gray')
    plt.show()
    
    plt.title('overlay crop')
    plt.imshow(overlay[my0:my1, mx0:mx1],)
    plt.show()
    
    
    truth = t
    num_sample = L
    zero_mv = 708-my0  #708 is the bottom of the dc pulse in image
    mV_per_pixel = 1/(2*GSY)
    
    resampled = mask_to_signal(crop, num_sample, zero_mv, mV_per_pixel)
    
    snr = compute_one_snr(resampled, truth, max_shift=100)
    print('snr', snr)
    
    plt.plot(resampled, label='resampled')
    plt.plot(t, label='truth')
    plt.legend()
    plt.show()